## 드라이브 마운트

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 라이브러리 설치

In [0]:
! pip install konlpy

In [0]:
! pip install kss

In [0]:
! pip install transformers

In [0]:
! pip install pytorch==0.1.2

  Using cached https://files.pythonhosted.org/packages/a9/41/4487bc23e3ac4d674943176f5aa309427b011e00607eb98899e9d951f67b/pytorch-0.1.2.tar.gz
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-41_2xjqu/pytorch/setup.py'"'"'; __file__='"'"'/tmp/pip-install-41_2xjqu/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-jc30idq5/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


## 데이터 불러오기

In [0]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

Using TensorFlow backend.


In [0]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import pickle
import kss

In [0]:
data_path = '/content/drive/My Drive/Colab Notebooks/LAB/국민청원/petition_data_all_2019_01_09.csv'
data = pd.read_csv(data_path)

In [0]:
data.head()

,num,category,start-days,end-days,person,progress,title,count,petition_overview
0,21,안전/환경,2017-08-19,2017-11-17,kakao - ***,청원종료,스텔라 데이지호에 대한 제안입니다.,9,\n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
1,22,기타,2017-08-19,2017-11-17,kakao - ***,청원종료,비리제보처를 만들어주세요.,17,\n\t\t\t\t\t\t\t\t\t\t\t현 정부에 국민들이 가장 원하는 것은 부...
2,23,미래,2017-08-19,2017-09-03,kakao - ***,청원종료,제2의 개성공단,0,\n\t\t\t\t\t\t\t\t\t\t\t만일 하시는 대통령님 및 각 부처 장관님...
3,24,일자리,2017-08-19,2017-08-26,kakao - ***,청원종료,공공기관 무조건적인 정규직전환을 반대합니다.,53,\n\t\t\t\t\t\t\t\t\t\t\t현정부에서 정규직 일자리를 늘리는 것에 ...
4,25,미래,2017-08-19,2017-09-03,kakao - ***,청원종료,제2의 개성공단,0,\n\t\t\t\t\t\t\t\t\t\t\t만일 하시는 대통령님 및 각 부처 장관님...


In [0]:
print(data.isnull().values.any())

False


In [0]:
len(data)

371230

In [0]:
data = data.drop_duplicates(subset='petition_overview', keep='last')

In [0]:
len(data)

352988

In [0]:
print(data.groupby('category').size().sort_values(ascending=False))

category
정치개혁           53258
기타             43175
인권/성평등         31114
안전/환경          27542
교통/건축/국토       25899
외교/통일/국방       24462
육아/교육          23557
보건복지           22330
일자리            20631
행정             18145
문화/예술/체육/언론    16571
미래             16114
경제민주화          15203
성장동력            6469
반려동물            3629
저출산/고령화대책       3199
농산어촌            1690
dtype: int64


In [0]:
category_selected = ['정치개혁', '인권/성평등', '안전/환경', '교통/건축/국토', '외교/통일/국방']

In [0]:
df = data[data['category'].isin(category_selected)]
df.head()

,num,category,start-days,end-days,person,progress,title,count,petition_overview
0,21,안전/환경,2017-08-19,2017-11-17,kakao - ***,청원종료,스텔라 데이지호에 대한 제안입니다.,9,\n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
6,27,정치개혁,2017-08-19,2017-09-18,naver - ***,청원종료,김이수 헌재소장 임명 재고 건의,1,\n\t\t\t\t\t\t\t\t\t\t\t문 대통령님께 묻습니다 (2017. 8....
12,33,인권/성평등,2017-08-19,2017-11-17,naver - ***,청원종료,납득할 수 있는 장애 재판정을 받고 싶습니다.,15,\n\t\t\t\t\t\t\t\t\t\t\t안녕하십니까? 저는 경직형 양마비 뇌병변...
14,35,외교/통일/국방,2017-08-19,2017-08-26,kakao - ***,청원종료,문재인정부는 신한국형 페리프로세스가 작성했습니다.,0,\n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 신한국형 페리프로세스 로...
15,36,인권/성평등,2017-08-19,2017-08-26,naver - ***,청원종료,한국채식인구 100만명. 학교 급식 및 군대에서 현미채식 선택권을 보장해주십시오!,1,\n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님과 각 정부 인사분들께 마...


In [0]:
X = df['petition_overview']

In [0]:
y = df['category']

In [0]:
X[:10]

0     \n\t\t\t\t\t\t\t\t\t\t\t스텔라 데이지호에 대한 제안입니다. \n...
6     \n\t\t\t\t\t\t\t\t\t\t\t문 대통령님께 묻습니다 (2017. 8....
12    \n\t\t\t\t\t\t\t\t\t\t\t안녕하십니까? 저는 경직형 양마비 뇌병변...
14    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 신한국형 페리프로세스 로...
15    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님과 각 정부 인사분들께 마...
17    \n\t\t\t\t\t\t\t\t\t\t\t문재인 대통령님 한중관계 사드 갈등 해소...
19    \n\t\t\t\t\t\t\t\t\t\t\t지금부터 67년전 1950년 6.25전쟁...
25    \n\t\t\t\t\t\t\t\t\t\t\t지금부터 67년전 1950년 6.25전쟁...
30    \n\t\t\t\t\t\t\t\t\t\t\t간절히 간절히 간청 드립니다. \n\n\...
54    \n\t\t\t\t\t\t\t\t\t\t\t※지장물관련 국토부의 해석 : 기존 건축...
Name: petition_overview, dtype: object

In [0]:
X = X.str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)

In [0]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

129820
32455
129820
32455


In [0]:
print(X_train.head())
print(y_train.head())

333598    봄에 이어 이번 가을 겨울에도 어김없이 미세먼지가 왔습니다 적어도 봄에는 마스크 안...
344228    부동산 계약 전산화 해야 합니다  부동산 계약 전산화 해야 합니다  부동산 계약 전...
300621    어제 뉴스를 보고 너무 어이가 없어서 올려봅니다 동료들한테 처음 들었을 때 너무 황...
285042    최근 국방의 의무에 관련하여 지속적으로 이야기가 나오고있습니다 국방의 의무에대하여 ...
26544                                          조두순 재심 해주세요 
Name: petition_overview, dtype: object
333598       안전/환경
344228    교통/건축/국토
300621      인권/성평등
285042    외교/통일/국방
26544        안전/환경
Name: category, dtype: object


In [0]:
def bert_add(body):
    lst = kss.split_sentences(body)
    result = ["[SEP]"] * (len(lst) * 2 + 1)
    result[1::2] = lst
    result[0] = "[CLS]"
    sentence = ' '.join(result)
    return sentence

In [0]:
label_dict = {'정치개혁': 0, '인권/성평등': 1, '안전/환경': 2, '교통/건축/국토': 3, '외교/통일/국방': 4}

In [0]:
X_train_bert = []
for body in X_train:
    X_train_bert.append(bert_add(body))

In [0]:
y_train_bert = []
for label in y_train:
    y_train_bert.append(label_dict[label])

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in X_train_bert]

print (X_train_bert[0])
print (tokenized_texts[0])

[CLS] 봄에 이어 이번 가을 겨울에도 어김없이 미세먼지가 왔습니다 [SEP] 적어도 봄에는 마스크 안써도 이정도까지는 아니였습니다 [SEP] 근데 지금 하늘이 뿌얗다못해 누렇습니다 [SEP] 게다가 마스크를 쓰는데도 불구하고 실내에 공기청정기를 켰는데도 불구하고 목이 계속 칼칼합니다 [SEP] 다음날 아침에는 목에 자꾸 가래가 생깁니다 [SEP] 미세먼지임에도 불구하고 누구는 목이 아픈게 환절기때문에 감기에 걸린것 같다고 합니다 [SEP] 또 밖이 뿌얗다보니 안개라고 오해하는 사람도 있었습니다 [SEP] 봄에는 미세먼지를 중국이 아닌 자동차 매연때문이라며 자가용 이용을 자제하고 대중교통을 이용하라고 교통비를 안받던적도 있었습니다 [SEP] 국민이 바보입니까 자동차 매연때문인지 중국에서 날라오는건지도 모를만큼요 지금 북한에 신경쓰고 계신건 알고 있습니다만 국민 먼저 살리는게 우선아닐까요 [SEP] 우리나라의 앞으로의 미래  먼 미래를 위해 앞으로 이나라에서 커갈 아이들을 위해 안전을 위해 그러시는건 알고있습니다 [SEP] 하지만 그 미래를 맞이하기도 전에 미세먼지 때문에 아플 아이들이 국민들이 있을 수 있습니다 [SEP] 건강해야 살아야 뭐든 좋게 맞이하지 않을까요 [SEP] 지금의 문제도 뒤돌아 봐주시길 바랍니다 [SEP] 미세먼지 해결 부탁드립니다 [SEP]
['[CLS]', '봄', '##에', '이어', '이', '##번', '가', '##을', '겨', '##울', '##에도', '어', '##김', '##없', '##이', '미', '##세', '##먼', '##지가', '왔', '##습', '##니다', '[SEP]', '적', '##어', '##도', '봄', '##에는', '마', '##스', '##크', '안', '##써', '##도', '이', '##정', '##도', '##까지', '##는', '아', '##니', '##였', '##습', '##니다', '[SEP]', '근', '##데', '지', '##금', '하', '##늘', '#

In [0]:
MAX_LEN = 256
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_ids[0]

array([   101,   9360,  10530,  64749,   9638,  35465,   8843,  10622,
         8877,  78123,  35979,   9546, 118667, 119136,  10739,   9309,
        24982, 118922,  80795,   9594, 119081,  48345,    102,   9664,
        12965,  12092,   9360,  15303,   9246,  12605,  20308,   9521,
        73131,  12092,   9638,  16605,  12092,  18382,  11018,   9519,
        25503, 119147, 119081,  48345,    102,   8926,  28911,   9706,
        40032,   9952, 118762,  10739,    100,   9032, 118871, 119081,
        48345,    102,   8872,  80244,   9246,  12605,  20308,  11513,
         9511,  41850,  12092, 103279,   9489,  31605,  10530,   8896,
        12310,  40311,  16605,  29669,   9811,  41850,  12092, 103279,
         9284,  10739,  77039,   9788, 119287,  33188,  48345,    102,
        52292,  41919,   9519, 119285,  15303,   9284,  10530,   9651,
       118694,   8843,  37388,  11287,   9420, 118668,  48345,    102,
         9309,  24982, 118922,  12508,  36240,  35979, 103279,   9032,
      

In [0]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [0]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    y_train_bert, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)


In [0]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

In [0]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9665,  24982,  13764,  40032,  14423,  52363,  10892,   8924,
         21711,  89523,  11261,   9489,  12424,  36553,  10622,  28195,   9670,
        119254,  58303,  48345,    102,   9978,   9665,  24982,  13764,  40032,
          9069,  52363,  10892,   9689, 119342,  13764,   8939,  28578,   9322,
         10622,   9460,  13767, 118624,  11261,   9524,  11664,   9647, 119081,
         48345,    102,   9689, 119342,  38939,   9881,  29669,   9960, 118991,
         13764,    100,   9551,  23811,  16439,   9955, 118671,  48549,    102,
          9638,   9670, 119254,  10892,   8924,  21711,  89523,  11261,   9448,
        118813,  40364,   8928,  15891,  48387,  19105,  79633,   9670, 119254,
         58303,  48345,    102,   9665,   9102,   9519,  87456,   9553,  23811,
         11261,   9663,  49515,   9711,  10892,   9256,  68773,  10739,  11261,
          9131,   9568, 118989,   9645,  48345,    102, 103596,   9465,  13764,
         19105,   9356,  14867,   9835, 

In [0]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
X_test_bert = []
for body in X_test:
    X_test_bert.append(bert_add(body))

In [0]:
y_test_bert = []
for label in y_test:
    y_test_bert.append(label_dict[label])

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in X_test_bert]

print (X_test_bert[0])
print (tokenized_texts[0])

[CLS] 주정차 위반한 차량을 구청소속 견인사무소에서 견인해갈때  왜 사전에 연락이 아예 없이 그대로 견인을 해가는지 저의가 궁금합니다 [SEP] 견인사무소에 가서 따져물어도 관련 근거 조례나 법령도 대지못하니 과태료및 견인비용을 수령하시는 분들은 욕한마디 먹지않고 수령하기는 거의 불가능하구요 [SEP] 연락을 하고 분정도뒤에 견인해가면  그나마 그분들이 욕도 안먹고 납부자들도 인정할수밖에없다고 생각합니다 [SEP] 아니면 정책을 만드시는 분들은 세금을 걷어야하고  세금을 수령하는 과정에서 수령기관자들이 욕먹는 것은 자신이 욕먹는게 아니니까 상관없다는 식으로 일임 하시는건가요 [SEP] 저의가 상당히 궁금합니다 [SEP] 왜 연락이나 통보 한마디 없이 견인해가는 속내가 뭡니까 [SEP]
['[CLS]', '주', '##정', '##차', '위', '##반', '##한', '차', '##량', '##을', '구', '##청', '##소', '##속', '견', '##인', '##사', '##무', '##소', '##에서', '견', '##인', '##해', '##갈', '##때', '왜', '사', '##전에', '연', '##락', '##이', '아', '##예', '없이', '그대로', '견', '##인을', '해', '##가는', '##지', '저', '##의', '##가', '궁', '##금', '##합', '##니다', '[SEP]', '견', '##인', '##사', '##무', '##소', '##에', '가', '##서', '따', '##져', '##물', '##어', '##도', '관련', '근', '##거', '조', '##례', '##나', '법', '##령', '##도', '대', '##지', '##못', '##하', '##니', '과', '##태', '##료', '##및', '견', '##인', '##비', '##용을', '수', '##령', '##하', '##시', '##는', '분', '##들은', '욕', '##한', '##마', '

In [0]:
MAX_LEN = 256

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9689,  16605,  23466,   9619,  30134,  11102,   9730,
        44321,  10622,   8908,  40311,  22333,  43962,   8880,  12030,
        12945,  32537,  22333,  11489,   8880,  12030,  14523, 101202,
       118832,   9596,   9405,  68767,   9568, 107693,  10739,   9519,
        96279,  73610, 110589,   8880,  83200,   9960,  68828,  12508,
         9663,  10459,  11287,   8916,  40032,  33188,  48345,    102,
         8880,  12030,  12945,  32537,  22333,  10530,   8843,  12424,
         9130,  41584,  29364,  12965,  12092,  86080,   8926,  41521,
         9678,  58762,  16439,   9341,  44220,  12092,   9069,  12508,
       118940,  35506,  25503,   8898,  83616,  38688, 118961,   8880,
        12030,  29455,  72444,   9460,  44220,  35506,  14040,  11018,
         9367,  22879,   9600,  11102,  23811,  48446,   9266,  12508,
       119112,  11664,   9460,  44220,  22440,  11018,  55067,   9368,
        11287,  74986,  35506,  17196,  48549,    102,   9568, 107693,
      

In [0]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,

In [0]:
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(y_test_bert)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9689,  16605,  23466,   9619,  30134,  11102,   9730,  44321,
         10622,   8908,  40311,  22333,  43962,   8880,  12030,  12945,  32537,
         22333,  11489,   8880,  12030,  14523, 101202, 118832,   9596,   9405,
         68767,   9568, 107693,  10739,   9519,  96279,  73610, 110589,   8880,
         83200,   9960,  68828,  12508,   9663,  10459,  11287,   8916,  40032,
         33188,  48345,    102,   8880,  12030,  12945,  32537,  22333,  10530,
          8843,  12424,   9130,  41584,  29364,  12965,  12092,  86080,   8926,
         41521,   9678,  58762,  16439,   9341,  44220,  12092,   9069,  12508,
        118940,  35506,  25503,   8898,  83616,  38688, 118961,   8880,  12030,
         29455,  72444,   9460,  44220,  35506,  14040,  11018,   9367,  22879,
          9600,  11102,  23811,  48446,   9266,  12508, 119112,  11664,   9460,
         44220,  22440,  11018,  55067,   9368,  11287,  74986,  35506,  17196,
         48549,    102,   9568, 107693, 

In [0]:
batch_size = 32

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [0]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [0]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 4

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [0]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


RuntimeError: ignored

In [0]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))